# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [60]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [61]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [62]:
wine_df.info()
wine_df['class'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  class

array([0, 1, 2])

#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [63]:
# Your answer here
# 178 rows x 14 cols
len(wine_df) #178 rows
print("Dataset contains: ",len(wine_df), "rows")

Dataset contains:  178 rows


_(ii)_ How many variables (columns) does the dataset contain?

In [64]:
# Your answer here
# 178 rows x 14 cols
#len(wine_df.columns) #14 columns
print("Dataset contains: ",len(wine_df.columns), "columns")

Dataset contains:  14 columns


_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [65]:
# Your answer here
wine_df["class"].info() # non-null int64 dtypes: int64(1)
wine_df["class"].unique() # array([0, 1, 2])
print("'variable type' of the response variable `class` : Categorical ")
print("'levels' (unique values) of the variable: 0,1 & 2")

<class 'pandas.core.series.Series'>
RangeIndex: 178 entries, 0 to 177
Series name: class
Non-Null Count  Dtype
--------------  -----
178 non-null    int64
dtypes: int64(1)
memory usage: 1.5 KB
'variable type' of the response variable `class` : Categorical 
'levels' (unique values) of the variable: 0,1 & 2



_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [66]:
# Your answer here
print(wine_df.columns.difference(['class']))
print("We have ",len(wine_df.columns.difference(['class'])), "predictor variables")

Index(['alcalinity_of_ash', 'alcohol', 'ash', 'color_intensity', 'flavanoids',
       'hue', 'magnesium', 'malic_acid', 'nonflavanoid_phenols',
       'od280/od315_of_diluted_wines', 'proanthocyanins', 'proline',
       'total_phenols'],
      dtype='object')
We have  13 predictor variables


You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [67]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  1.518613   -0.562250  0.232053          -1.169593   1.913905   
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.808997    1.034819             -0.659563         1.224884   
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

It is important to standardize the predictor variables as 
1. It has direct impact on prediction.
2. We scale our variables to avoid model overvaluing variables on a larger scale or undervaluing variables that are on smaller scale.
3. We scale & centre our variables to make sure each variable contributes equally to the prediction calculation as even small scale variables might be as meaningful or even more meaningful to the accuracy precision in predictions.

(ii) Why did we elect not to standard our response variable `Class`?

Response variable class identifies each wine. It has only 3 unique values of 0, 1 and 2 and is used for prediction of this response variable class only.

(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

Setting a seed value is important to ensure that the sequence of random numbers is reproducible. It essentially produces same random result everytime we run our code. So particular seed value guarantees same results each time the analysis is run. Different seed values result in different random patterns, but the same values ensures consistent results.

(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [68]:
# set a seed for reproducibility
np.random.seed(123)

# split the data into a training and testing set. hint: use train_test_split !
predictors_train, predictors_test,y_train,y_test = train_test_split(
    predictors_standardized, wine_df['class'],train_size=0.75, shuffle=True
)
# Your code here ...
#predictors_train # 133 rows
#predictors_test # 45 rows

#predictors_train.info()
#predictors_train['class'] = wine_df['class'] # Added Class response variable back to training set for fitting y
#predictors_test['class'] = wine_df['class'] # Added Class response variable back to testing set 
#predictors_train
#predictors_test

predictors_train.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
28,1.073917,-0.391694,1.584529,-0.028510,0.509638,1.049347,0.944462,0.065639,0.296279,-0.241413,1.283518,1.113449,0.535335
13,2.160950,-0.544297,0.085839,-2.430790,-0.613775,1.289697,1.667318,0.549108,2.135968,0.147900,1.283518,0.167113,1.283691
88,-1.680724,-0.248068,0.341713,0.632117,-1.105268,-0.552984,-0.340615,0.951998,-0.422075,-0.976782,0.186684,0.195361,-0.213021
24,0.616869,-0.472483,0.890014,0.151661,-0.262708,0.376368,0.583034,-0.659563,0.121071,-0.665332,0.713164,1.706675,0.312420
100,-1.137207,-0.230114,-2.436346,-0.599052,-0.192495,-0.104332,0.141289,-0.820719,-0.334471,-0.760497,1.371265,0.491974,-0.117486


#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [69]:
# Your code here...
knn = KNeighborsClassifier()
parameter_grid = {
    "n_neighbors": range(1, 51)
}

predictors_tune_grid = GridSearchCV(
    estimator=knn,
    param_grid=parameter_grid,
    cv=10
)

predictors_tune_grid.fit(
    predictors_train[["alcohol", "malic_acid","ash","alcalinity_of_ash","magnesium","total_phenols","flavanoids","nonflavanoid_phenols","proanthocyanins","color_intensity","hue","od280/od315_of_diluted_wines","proline"]],
   # predictors_train['class']
   y_train
)

print("The best value for n_neighbors based on grid search results",predictors_tune_grid.best_params_['n_neighbors'])

# n_neighbors = 15

The best value for n_neighbors based on grid search results 15


#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [ ]:
# Your code here...
knn = KNeighborsClassifier(n_neighbors=predictors_tune_grid.best_params_['n_neighbors'])
knn.fit(X=predictors_train[["alcohol", "malic_acid","ash","alcalinity_of_ash","magnesium","total_phenols","flavanoids","nonflavanoid_phenols","proanthocyanins","color_intensity","hue","od280/od315_of_diluted_wines","proline"]], 
        #y=predictors_train["class"]
        y=y_train)

predictors_test["predicted"] = knn.predict(predictors_test[["alcohol", "malic_acid","ash","alcalinity_of_ash","magnesium","total_phenols","flavanoids","nonflavanoid_phenols","proanthocyanins","color_intensity","hue","od280/od315_of_diluted_wines","proline"]])
predictors_test

accuracy = knn.score(
    predictors_test[["alcohol", "malic_acid","ash","alcalinity_of_ash","magnesium","total_phenols","flavanoids","nonflavanoid_phenols","proanthocyanins","color_intensity","hue","od280/od315_of_diluted_wines","proline"]],
    #predictors_test["class"]
    y_test
)
print("Accuracy Score: :",accuracy)
# 93.33 accuracy

accuracy_grid_score = pd.DataFrame(predictors_tune_grid.cv_results_)

accuracy_grid_score.sort_values(by=['rank_test_score','param_n_neighbors'])
#Rank 1 for n_neighbors=15 

Accuracy Score: : 0.9333333333333333


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
14,0.000334,0.000028,0.000581,0.000041,15,{'n_neighbors': 15},1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.923077,0.923077,0.984615,0.030769,1
18,0.000327,0.000015,0.000587,0.000023,19,{'n_neighbors': 19},1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.923077,0.923077,0.984615,0.030769,1
19,0.000328,0.000022,0.000620,0.000098,20,{'n_neighbors': 20},1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.923077,0.923077,0.984615,0.030769,1
20,0.000319,0.000010,0.000589,0.000052,21,{'n_neighbors': 21},1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.923077,0.923077,0.984615,0.030769,1
21,0.000358,0.000102,0.000566,0.000024,22,{'n_neighbors': 22},1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.923077,0.923077,0.984615,0.030769,1
22,0.000316,0.000017,0.000576,0.000048,23,{'n_neighbors': 23},1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.923077,0.923077,0.984615,0.030769,1
23,0.000328,0.000032,0.000569,0.000031,24,{'n_neighbors': 24},1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.923077,0.923077,0.984615,0.030769,1
24,0.000320,0.000026,0.000583,0.000031,25,{'n_neighbors': 25},1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.923077,0.923077,0.984615,0.030769,1
25,0.000333,0.000030,0.000597,0.000046,26,{'n_neighbors': 26},1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.923077,0.923077,0.984615,0.030769,1
27,0.000318,0.000010,0.000589,0.000059,28,{'n_neighbors': 28},1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.923077,0.923077,0.984615,0.030769,1


# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `11:59 PM - 11/24/2025`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/LCR/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#dc2-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
